In [3]:
from dotenv import load_dotenv
from os import getenv
import json, requests
load_dotenv()

True

In [4]:
CERT_URI = r'https://api.cert.tastyworks.com'
CERT_WSS = r'wss://streamer.cert.tastyworks.com'

PROD_URI = r'https://api.tastyworks.com'
PROD_WSS = r'wss://streamer.tastyworks.com'

In [13]:
class TTApi:
    headers: dict = {}

    def __init__(self) -> None:
        self.headers["Content-Type"] = "application/json"
        self.headers["Accept"] = "application/json"
    def __post(
            self, endpoint:str = None, body:dict={}, headers:dict=None
    ) -> requests.Response:
        if headers is None:
            headers = self.headers

        response = requests.post(
            self.tt_uri + endpoint,
            data=json.dumps(body),
            headers=headers
        )

        if response.status_code == 201:
            return response.json()
        
        print(f"Error {response.status_code}")
        print(f"Endpoint: {endpoint}")
        print(f"Body: {body}")
        print(f"Headers: {headers}")
        print(f"Response: {response.text}")
        return None


    def login(self, username, password, mfa:str|None = None) -> bool:
        body = {
            'login':username,
            'password':password,
            'remember-me':True
        }
        
        if mfa is not None:
            self.headers['X-Tastyworks-OTP'] = mfa
            self.tt_uri = PROD_URI
            self.wss_uri = PROD_WSS

        else:
            self.tt_uri = CERT_URI
            self.wss_uri = CERT_WSS

        response = self.__post('/sessions', body=body, headers=self.headers)
        if response is None:
            return False
        
        self.user_data = response['data']['user']
        self.session_token = response['data']['session-token']
        self.headers['Authorization'] = self.session_token

        if mfa is not None:
            del self.headers['X-Tastyworks-OTP']

In [14]:
api = TTApi()
api.login(getenv('username'), getenv('password'))

AttributeError: 'TTApi' object has no attribute 'username'